In [ ]:
from google.colab import files
files.upload()  # upload kaggle.json here

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rakeshkapilavai/extrovert-vs-introvert-behavior-data")

print("Path to dataset files:", path)

In [ ]:
import os

path = "/kaggle/input/extrovert-vs-introvert-behavior-data"
print("Files in dataset folder:", os.listdir(path))

In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/extrovert-vs-introvert-behavior-data/personality_dataset.csv')

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
df['Stage_fear'] = df['Stage_fear'].replace({'Yes': 1, 'No': 0})
df['Drained_after_socializing'] = df['Drained_after_socializing'].replace({'Yes': 1, 'No': 0})
df['Personality'] = df['Personality'].replace({'Extrovert': 1, 'Introvert': 0})

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
df.corr()

In [ ]:
positive_corr_cols = [
    'Social_event_attendance',
    'Going_outside',
    'Post_frequency',
    'Friends_circle_size'
]

df_selected = pd.concat([df[positive_corr_cols], df['Personality']], axis=1)

In [ ]:
df_selected

In [ ]:
df_selected['Personality'].value_counts()

In [ ]:
df_selected.isnull().sum()

In [ ]:
from sklearn.impute import SimpleImputer

si = SimpleImputer(strategy='mean')

df_imputed = si.fit_transform(df_selected[positive_corr_cols])

df_imputed = pd.DataFrame(df_imputed, columns=positive_corr_cols)

In [ ]:
final_df = pd.concat([df_imputed, df_selected['Personality']], axis=1)

In [ ]:
final_df

In [ ]:
final_df.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.boxplot(final_df['Friends_circle_size'])

In [ ]:
sns.boxplot(final_df['Post_frequency'])

In [ ]:
sns.boxplot(final_df['Going_outside'])

In [ ]:
sns.boxplot(df['Social_event_attendance'])

## splitting training and testing data

In [ ]:
from sklearn.model_selection import train_test_split

X = final_df.drop('Personality', axis=1)
y = final_df['Personality']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## model training and prediction

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, accuracy_score, classification_report, confusion_matrix

rfc = RandomForestClassifier(n_estimators=150)

rfc.fit(X_train, y_train)

In [ ]:
y_pred = rfc.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
TN, FP, FN, TP = cm.ravel()

In [ ]:
TN, FP, FN, TP

In [ ]:
accuracy = (TP + TN) / (TN+ FP+ FN+ TP)
print(accuracy)

In [ ]:
Precision = TP / (TP + FP)
print(Precision)

## query points

In [ ]:
query_pt = [[0, 2, 0, 2]]

In [ ]:
rfc.predict(query_pt)   # corret - Abhiraj's friend

In [ ]:
query_pt2 = [[0, 2, 2, 5]]

In [ ]:
rfc.predict(query_pt2)   # correct

In [ ]:
query_pt3 = [[2, 6, 4, 14]]

In [ ]:
rfc.predict(query_pt3)     # correct

In [ ]:
query_pt4 = [[1, 3, 2, 5]]
rfc.predict(query_pt4)    # wrong

In [ ]:
query_pt5 = [[1, 4, 2, 9]]
rfc.predict(query_pt5)     # correct

In [ ]:
query_pt6 = [[2, 3, 1, 3]]
rfc.predict(query_pt6)     # correct

In [ ]:
query_pt7 = [[2, 2, 3, 20]]
rfc.predict(query_pt7)     # correct

In [ ]:
query_pt8 = [[3, 3, 3, 6]]
rfc.predict(query_pt8)      # wrong

In [ ]:
query_pt9 = [[3, 3, 0, 3]]
rfc.predict(query_pt9)      #

In [ ]:
import joblib
joblib.dump(rfc, "personality.pkl")